# **LINEARISATION UNIQUE DES TRAFICS SUR BORDEAUX METROPOLE**
> ATTENTION : IL Y A DES MELI-MELO ENTRE LE FICHIER gdf_rhv_groupe qui contient un idtronc pour tout les troncons et a été constitué avant de prendre en compte les cat_rhv 62, 63 etc...; qui est utilisé dans toute la partie affectation points de comptage, et le fichier issus de la Bdd graph_filaire qui lui contient bien toute les lignes mais n'a pas d'idtronc renseigné.<br> il faudrait remettre tout ça à plat

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
from shapely import speedups #gestion erreur GEOSGeom_createLineString_r returned a NULL pointer
speedups.disable()
from shapely.geometry import MultiPoint, Polygon, box, Point
from shapely.ops import nearest_points
from shapely.wkt import dumps, loads
import re, json
from unidecode import unidecode
from copy import copy, deepcopy
from difflib import get_close_matches,SequenceMatcher
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from collections import Counter
from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu,verif_index, gp_changer_nom_geom

from Simplifier_Rdpt import creer_dico_noeud_rdpt,simplifier_noeud_rdpt,maj_graph_rdpt, donnees_tot_rd_pt
import Estim_trafic as et
import Comptage as Cpt
import Affectation_pt_comptage as ap
import Voies_Cat4 as v4
from MMM import import_fichiers_mmm

pd.set_option('display.max_columns', 100)
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', 50)


from Base_BdTopo.Import_outils import import_donnes_base
from Base_BdTopo.Rond_points import identifier_rd_pt
from Base_BdTopo.Regroupement_correspondance import *
from Base_BdTopo.Troncon_elementaire import *
from Base_BdTopo.Troncon_base import *
from Base_BdTopo.Gestion_2_chaussee import *
import plotly.express as px
import altair as alt

from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

## 1 Regrouper les troncons du filaire de voie
>Les troncons du filaire de voie sont regroupés d'abord uniquement avec les troncons de cat_rhv égal à 1,2,3,61,62,63, pour pouvoir réaliser des troncon de trafic homogènes tous reliés entre eux.
<br> Ensuite, un deuxième regroupement comprenant l'ensemble des troncons de cat_rhv permetd'affiner le diagnostic

> ### 1.1 Import du fichier final
le fichier shape est vérifié à la main car l'affectation automatique n'est pas 100% fiable

In [2]:
# Pour info : lire le graph issu de la methode decrite dessous
with ct.ConnexionBdd('local_otv', localisation='maison') as c :
    graph_filaire_123 = gp.read_postgis('select * from linearisation_bm.graph_rhv_123_v2', c.connexionPsy)
    graph_filaire_123_vertex = gp.read_postgis('select id,cnt,chk,ein,eout,the_geom as geom from linearisation_bm.graph_rhv_123_v2_vertices_pgr', c.connexionPsy)
    graph_filaire = gp.read_postgis('select * from linearisation_bm.graph_rhv_complet', c.connexionPsy)
    graph_filaire_vertex=gp.read_postgis('select id,cnt,chk,ein,eout,the_geom as geom from linearisation_bm.graph_rhv_complet_vertices_pgr', c.connexionPsy)
gdf_rhv_groupe_123=graph_filaire_123
#gdf_rhv_groupe_123=gp.read_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123_v2.shp').merge(graph_filaire_123[['ident','source','target']], on='ident', how='left')"""
gdf_rhv_groupe=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp.shp').merge(graph_filaire[['ident','source','target']], on='ident', how='left')

>### 1.2 Méthode
 pour information

In [ ]:
# 1. import du fichier de filaire de base
gdf_rhv=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\Filaire_voie\FV_TRONC_L.shp')
# 2. mise en forme
gdf_rhv.columns=[a.lower() for a in gdf_rhv.columns] #nom de colonne en minuscule
gdf_rhv.rename(columns={'gid':'id','nom_voie' : 'numero'},inplace=True)
gdf_rhv=gdf_rhv.loc[~gdf_rhv.id.isna()].copy()
gdf_rhv['id_ign']=gdf_rhv.ident.apply(lambda x : 'TRONROUT'+str(x))
gdf_rhv=gdf_rhv.loc[~gdf_rhv.cat_rhv.isin(['98','99'])].copy()
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Route à 2 chaussées' if x['rgraph_dbl']==0 else 'Route à 1 chaussée', axis=1)
gdf_rhv['nature']=gdf_rhv.apply(lambda x : 'Bretelle' if x['cat_dig']=='8' else x['nature'], axis=1)
gdf_rhv['numero']=gdf_rhv.apply(lambda x : 'Bretelle '+x['numero'] if x['cat_dig']=='8' else x['numero'], axis=1)
gdf_rhv['sens']=gdf_rhv.apply(lambda x : 'Direct' if x['rgraph_dbl']==0 else 'Double', axis=1)
gdf_rhv['codevoie_d']='NR'
gdf_rhv['importance']=gdf_rhv['cat_dig']
gdf_rhv['id']=gdf_rhv.id.apply(lambda x : int(x))
#gdf_rhv.loc[gdf_rhv.nom_voie.isna()] #verif des nom_voie null : 2 lignes, cf traitements plus loins
#filtrer les voies cylcables et autres
gdf_rhv_filtre=gdf_rhv.loc[~gdf_rhv.cat_dig.isin(['6','7','9','10'])].copy()

# 3. creer le graph en bdd
#creer_graph(gdf_rhv_filtre.loc[gdf_rhv_filtre.cat_rhv.isin(['1','2','3','33','61','62','63'])],
            #'local_otv',schema='linearisation_bm',table='graph_rhv_123',table_vertex='graph_rhv_123_vertices_pgr')
creer_graph(gdf_rhv_filtre,'local_otv',schema='linearisation_bm',table='graph_rhv_complet',table_vertex='graph_rhv_complet_vertices_pgr')

In [ ]:
# 4. initialiser les paramètre de regroupement
df=import_donnes_base('local_otv','linearisation_bm', 'graph_rhv_complet','graph_rhv_complet_vertices_pgr')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']=='Bretelle'].copy()
bretelle_tri=bretelle.loc[bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
sans_bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']!='Bretelle'].copy()
list_sans_bretelle=sans_bretelle.id_ign.tolist()
list_tri_longueur=sans_bretelle.loc[sans_bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
list_id_ign=bretelle_tri+list_tri_longueur

In [ ]:
# 5. appel fonction de grouepement pour creer des idtronc pour uniquement voies de 1 à 3 et d'un autre coté pour toute les voies
df_affectation, dico_erreur, lignes_traitees, lignes_non_traitees=regrouper_troncon(
    list_id_ign, df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])

# 6. mise en forme et export
gdf_rhv_groupe=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
gdf_rhv_groupe.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\rhv_grp_v2.shp')
#gdf_rhv_groupe_123=gdf_rhv_filtre.merge(df_affectation, left_on='id_ign', right_on='id', how='left')
#gdf_rhv_groupe_123.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_groupe_123.shp')

## 1. MISE EN FORME DES COMPTAGES PONCTUELS
Pour ces comptages l'idée c'est de réussir à regrouper les comptages par un idtronc issu de la detection des troncon elementaire. ça a du sens car pour un comptage ponctuel normalement les 2 sens de cicru sont sur un troncon unique. DE plus, les ens circu, noms de voie et autres attributs permettant de regrouper les comptages sont de qualité trop variable pour s'appuyer uniquement dessus. et enfin, peu de 2*2 vois sont meusrés

In [6]:
affect_finale,cpt_pct_l93=Cpt.traitements_bdxm_pct(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\base_comptages_routiers_finale.xlsx',gdf_rhv_groupe)

## 2. MISE EN FORME DES COMPTAGES PERMANENTS
Pour ces compteurs on va regrouper en séparant les siredo des boucles liées au système gertrude. Pour les siredo pas de pb, pour les autre sc'est plus compliqué car les références de localiusation varient parfois. Il y a donc une phase de regroupement manuel à la fin

> ### 2.1 Import du fichier final

In [7]:
#import fichier corrige_final
cpt_perm_final=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

> ### 2.1 Méthode
Pour info

In [ ]:
#creer un gdf avec la fusion des points et des données de comptages non nulles
cpt_brut=pd.read_excel(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\comptage_trafic_final.xls')
cpt_brut.columns=[a.lower() for a in cpt_brut.columns]
cpt_brut=cpt_brut.loc[~cpt_brut.mjo_val.isna()].copy()
pt_cpt_perm=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\PC_CAPTE_P.shp')
pt_cpt_perm.columns=[a.lower() for a in pt_cpt_perm.columns]
pt_cpt_perm=pt_cpt_perm.merge(cpt_brut, on='ident')
pt_cpt_perm['nom_voie']=pt_cpt_perm.nom_voie.apply(lambda x : re.sub(('é|è|ê'),'e',x.lower().replace('  ',' ')))#mise en forme nom de voie

#regrouper les comptages
#un compteur peut caracteriser 1 ligne, ou 2 compteur 1 ligne ou 2 compteurs 2 lignes, dc les compteurs ne vont pas toujours par deux

#regrouper geographiquement
pt_cpt_perm['x_l93']=pt_cpt_perm.geometry.apply(lambda x : x.x)
pt_cpt_perm['y_l93']=pt_cpt_perm.geometry.apply(lambda x : x.y)
data_test_clust=[[x, y] for x, y in zip(pt_cpt_perm.x_l93.tolist(), pt_cpt_perm.y_l93.tolist())]
db = DBSCAN(eps=200, min_samples=2).fit(data_test_clust)
labels = db.labels_
pt_cpt_perm['n_cluster']=labels

#correspondance nom_rue
#analyser les noms de commune : 
#grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]).unique()#liste des permiers mots
#grp_nom_voie.loc[grp_nom_voie.nom_voie.apply(lambda x : x.split()[0]=='carbon')].nom_voie.unique() #test des valeusr
dico_commune={'le bouscat','le haillan','bordeaux','pessac','talence', 'merignac','st medard','carbon blanc',
       'begles', 'eysines','villenave d\'ornon','bruges','gradignan','bassens','cenon','floirac','pessa','lormont', 'artigues'}
dico_type_voie={'allee' : ['all.','allee', 'allees'],'avenue': ['av', 'av.', 'avenue'], 'boulevard' : ['blvd'],
               'bretelle' : 'bretelle', 'cours' : ['crs','cours'], 'cote' : ['côte'], 'place':['pl'], 'route':['route', 'rte'],'rue':['rue'],
               'voie':['voie']}
def decoupe_nom_voie(nom_voie) : 
    for c in dico_commune : 
        if nom_voie[:len(c)]==c :
            commune=nom_voie[:len(c)]
            nom_voie=nom_voie[len(c):].strip()
    for k, v in dico_type_voie.items() : 
        if any([a for a in nom_voie.split() if a in v])  : 
            caractere=[a for a in nom_voie.split() if a in v][0]
            type_voie=k
            nom_voie=' '.join([n for n in nom_voie.split() if n!=caractere])
            break
    else : type_voie=None
    for test in ['avant','apres'] :
        if test in nom_voie : 
            return nom_voie.split(test)[0].strip(), test,nom_voie.split(test)[1].strip(), commune, type_voie
    else : return nom_voie, nom_voie,nom_voie,commune,type_voie

pt_cpt_perm['nom_rue']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[0])
pt_cpt_perm['localisant']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[1])
pt_cpt_perm['rue_refer']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[2])
pt_cpt_perm['commune']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[3])
pt_cpt_perm['type_voie']=pt_cpt_perm.nom_voie.apply(lambda x : decoupe_nom_voie(x)[4])
pt_cpt_perm.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\pt_cpt_perm.shp')

>>#### 2.2.1 SIREDO

In [ ]:
siredo=pt_cpt_perm.loc[pt_cpt_perm['type']=='SIREDO'].copy()#isoler siredo
cross_join=siredo.assign(key=1).merge(siredo.assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join_filtre=cross_join[[c for c in cross_join.columns if c[-2:]=='_x']+['distance','ident_y']].rename(columns=
              {c : c[:-2] for c in cross_join.columns if c[-2:]=='_x'}).copy()#filtre des attribut
cross_join_filtre=cross_join_filtre.loc[(cross_join_filtre['ident']!=cross_join_filtre['ident_y'])].copy()
siredo_proches=cross_join_filtre.loc[(cross_join_filtre.groupby('ident')['distance'].transform(min)==
                                      cross_join_filtre['distance'])].copy()#plus proche voisin
siredo_proches['id_grpsiredo']=siredo_proches.reset_index().index
id_siredo_proches=(pd.concat([siredo_proches[['ident','ident_y', 'id_grpsiredo']],
           siredo_proches[['ident_y','ident', 'id_grpsiredo']].rename(columns={
    'ident_y':'ident', 'ident':'ident_y'})],sort=False).sort_values('ident').
    reset_index().drop_duplicates('index').drop_duplicates(['ident','ident_y']).drop('index',axis=1))
cle_assoc_siredo=pd.concat([id_siredo_proches[['ident','id_grpsiredo']],id_siredo_proches[['ident_y','id_grpsiredo']].rename(columns={
    'ident_y':'ident'})]).sort_values('id_grpsiredo')

>>#### 2.2.2 Gertrude
> au final ce n'est pas très efficace, et un renseignement des regroupement a la ain aurait été plus rapide. il sera important si de nouvelles stations sont posées de fournir de suite un identifiant de regroupement

In [ ]:
gertrude=pt_cpt_perm.loc[pt_cpt_perm['type']=='BOUCLE'].copy()#isoler les boucles
gertrude['x']=gertrude.geometry.apply(lambda x : x.x)
gertrude['y']=gertrude.geometry.apply(lambda x : x.y)

In [ ]:
#si c'est sur le mm idtronc on regroupe

#rappatrier le numero d'id_tronc pour chaque comptage
#trouver la distance min à chaque objet ligne"du rhv groupe
grp_troncon_temp=gertrude.copy()
grp_troncon_temp.geometry=grp_troncon_temp.buffer(20)#passer la geom en buffer
intersct_buff_20=gp.sjoin(grp_troncon_temp,gdf_rhv_groupe,how='left',op='intersects')
intersct_buff_20.geometry=gertrude.geometry#repasser la geom en point
intersct_buff_20=intersct_buff_20.merge(gdf_rhv_groupe[['ident','geometry']], left_on='ident_right', right_on='ident')
intersct_buff_20['dist_pt_ligne']=intersct_buff_20.apply(lambda x : x['geometry_x'].distance(x['geometry_y']), axis=1) #définir la disance entre les points et ligne
joint_dist_min=intersct_buff_20.loc[intersct_buff_20.groupby('ident_left')['dist_pt_ligne'].transform(min)==intersct_buff_20['dist_pt_ligne']][['ident_left','idtronc','numero']].rename(
    columns={'ident_left':'ident'}).copy()#ne garder que la ligne la plus proche
grp_troncon=gertrude.merge(joint_dist_min, on='ident',how='left')#df finale

#regrouper
grp_idtronc=grp_troncon.groupby('idtronc')['ident'].agg(lambda x : tuple(x))
grp_idtronc=grp_idtronc.loc[grp_idtronc.apply(lambda x : len(x)>1)].reset_index().copy()
grp_idtronc['id_grp']=grp_idtronc.ident.rank()
#mettre en forme
grp_idtronc=pd.DataFrame([(a,g) for i,g in zip(grp_idtronc.ident.tolist(),grp_idtronc.id_grp.tolist()) for a in i], columns=['ident','id_grp'])
grp_final=grp_idtronc.copy()

In [ ]:
# trouver les points qui restent et préparer un regrouepement par commune, nom de rue, distance et sens

#points qui restent
gertrude_etape2=gertrude.loc[~gertrude['ident'].isin(grp_idtronc.ident.tolist())].copy()
#comparer les noms
cross_join=gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1).merge(
    gertrude_etape2[['gid','geometry','ident','nom_voie', 'sens_cir','commune', 'nom_rue', 'localisant','rue_refer','type_voie']].assign(key=1), on='key').drop('key',axis=1)#calcul des distances
cross_join['distance']=cross_join.apply(lambda x : x['geometry_x'].distance(x['geometry_y']),axis=1)
cross_join['comp_nom_rue']=cross_join.apply(lambda x : SequenceMatcher(None,x['nom_rue_x'], x['nom_rue_y']).ratio(), axis=1)#affecter une note a cahque relation

#vérifier que les sens de cicru sont cohérents et ne garder que les lignes avec un sens ok, un nom de voie proche et une distance proche
gertrude_join=cross_join.loc[(cross_join['comp_nom_rue']>0.6) & (cross_join['distance']<1000) & (cross_join['commune_x']==cross_join['commune_y']) & 
                             (cross_join['type_voie_x']==cross_join['type_voie_y'])].copy()
#on ajoute un attribut de verif du sens
def verif_sens(s1,s2) : 
    for sens in [['Sens Sortant', 'Sens Entrant'],['Sens N > S', 'Sens S > N'], ['Sens Nord > Sud', 'Sens Sud > Nord'],['Nord > Sud', 'Sud > Nord'],
                ['Boulevards Nord > Sud', 'Boulevards Sud > Nord']] :
        if s1 in sens and s2 in sens : 
            if s1!=s2 : 
                return True
        else : 
            continue
    return False
gertrude_join['valid_sens']=gertrude_join.apply(lambda x : verif_sens(x['sens_cir_x'],x['sens_cir_y']),axis=1)  
gertrude_join['id_grp']=gertrude_join.sort_values('ident_x').ident_x.rank(method="min")
gertrude_grp=gertrude_join.loc[gertrude_join['valid_sens']][['ident_x','ident_y','distance', 'comp_nom_rue', 'valid_sens','id_grp']].sort_values('ident_x')

In [ ]:
#obtenir une liste des ident qui se regroupent
list_id_grp=pd.concat([gertrude_grp[['ident_x','id_grp']],gertrude_grp[['ident_y','id_grp']].rename(columns={'ident_y' : 'ident_x'})]).drop_duplicates([
    'ident_x','id_grp']).groupby('id_grp').ident_x.agg(lambda x : set(sorted(tuple(x)))).tolist()
dico={}
for e,c in enumerate(gertrude_grp.ident_x.tolist()) : 
    for l in list_id_grp : 
        if c in l :
            #print('c dans l','c : ',c,'l :',l)
            if c not in dico.keys() :
                dico[c]=list(l)
            else : 
                dico[c]+=list(l)
list_id_grp_temp=set([ v for v in {k : tuple(set(sorted(dico[k]))) for k in dico.keys()}.values()])
list_id_grp_final=set(v for v in {c :b for c in gertrude_grp.ident_x.tolist() 
 for b in [a for a in list_id_grp_temp if c in a] 
 if len(b) == max([len(c) for c in [a for a in list_id_grp_temp if c in a]])}.values())

In [ ]:
#affecter un identifiant
def groupe_pt_gertrude_etape2(df_local) : 
    """
    regrouper les points à partir d'une liste d'ident selon l'ordre d'apparition 
    """
    limite_box=MultiPoint([(a.x,a.y) for a in df_local.itertuples()]).bounds
    plg_long, plg_larg=limite_box[2]-limite_box[0],limite_box[3]-limite_box[1]
    if max(plg_long,plg_larg)==plg_long : 
        df_local['next_sens']=df_local.sort_values('x').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('x').sens_cir.shift(1,fill_value='NC')
    else : 
        df_local['next_sens']=df_local.sort_values('y').sens_cir.shift(-1,fill_value='NC')
        df_local['prev_sens']=df_local.sort_values('y').sens_cir.shift(1,fill_value='NC')
    df_local['sens_comp']=df_local.apply(lambda x : x.prev_sens if x.next_sens=='NC' else x.next_sens,axis=1)
    df_local['verif_sens']=df_local.apply(lambda x : verif_sens(x['sens_cir'],x['sens_comp']),axis=1)
    df_local['id_grp']=-99
    i=0
    while i < len(df_local)-1:
        if df_local.iloc[i].verif_sens : 
            df_local.loc[df_local.index.isin(df_local.iloc[i:i+2].index.tolist()),'id_grp']=i
            i+=2
        else : 
            i+=1
    return

# à partior de la liste des ident groupé : 
grp_final2=grp_final.copy()
# on selectionne ces points dans la df
increment=grp_final2.id_grp.max()
for l in list_id_grp_final : 
    #if 'Z8CT13' not in l : continue
    df_local=gertrude_etape2.loc[gertrude_etape2.ident.isin(l)].copy()
    if len(df_local)==1:
        continue
    if len(df_local)==2 : 
        if (gertrude_join.loc[(gertrude_join.ident_x.isin(l)) & (gertrude_join['ident_x'] != gertrude_join['ident_y'])].valid_sens).all() : 
            increment+=1
            df_local['id_grp']=increment
            grp_final2=pd.concat([grp_final2,df_local[['ident','id_grp']]],sort=False)
    else :
        groupe_pt_gertrude_etape2(df_local)
        increment+=grp_final2.id_grp.max()
        df_local.loc[df_local['id_grp']!=-99,'id_grp']=df_local['id_grp']+increment
        grp_final2=pd.concat([df_local[['ident','id_grp']],grp_final2],sort=False)

In [ ]:
#rappatrier l'id_grp sur les données de base et export pour modif manuelle
gertrude.merge(grp_final2, on='ident',how='left').to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')

#import des modifs manuelles et mise en forme finale
gertrude_manu=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude.shp')
gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp']=gertrude_manu.loc[(gertrude_manu.id_grp.isna()) | (gertrude_manu.id_grp==-99),'id_grp'].reset_index().index + gertrude_manu.id_grp.max()
gertrude_manu.loc[gertrude_manu.type_grp.isnull(),'type_grp']='auto'
#export pour dernière modif manuelle
gertrude_manu.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')

#fichier final
gertrude_fin=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gertrude_final.shp')
#concat avec les siredo
cle_assoc_siredo['id_grpsiredo']=cle_assoc_siredo['id_grpsiredo']+gertrude_fin.id_grp.max()
cpt_perm_final=pd.concat([gertrude_fin,siredo.merge(cle_assoc_siredo, on='ident').rename(columns={'id_grpsiredo':'id_grp'})],axis=0, sort=False)
cpt_perm_final.to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\cpt_perm_groupe.shp')

# 3. Affecter du trafic au filaire de voie

> ### On commence par affecter le trafic aux troncons des catégories 1,2,3 à partirdes comptages permanents.<br> Pour le détail des fonctions cf module Affectation_pt_comptage

In [8]:
gdf_rhv_cpt_perm_123,lgn_proche_perm=ap.affectation_cpt_perm(cpt_perm_final,gdf_rhv_groupe,gdf_rhv_groupe_123,10)
#ap.export_cpt_perm_linearises(gdf_rhv_cpt_perm_123,r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_cpt_perm.shp')

> ### ensuite les comptages ponctuels

In [9]:
#affectation des comptages ponctuels et exports
gdf_traf_tot=ap.affectation_cpt_ponct(cpt_pct_l93, affect_finale, lgn_proche_perm, gdf_rhv_groupe, 10, gdf_rhv_groupe_123, gdf_rhv_cpt_perm_123)
#ap.export_cpt_ponct_linearises(gdf_traf_tot,r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_cpt_perm-ponct.shp')

## 2. MMM
En premier lieu il faut mettre à jour les attributs en renomant de façon explicite puis en faisant la somme des 2 sens de circulation pour les voies doucble sens cf module MMM

In [ ]:
fichier_src_simpl=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019.shp')

In [ ]:
#creer graph et importer 
creer_graph(fichier_src_simpl, 'local_otv',id_name='NO', schema='linearisation_bm', table='mmm_graph_2019', table_vertex='mmm_graph_2019_vertices_pgr')

In [ ]:
#importer le graph
with ct.ConnexionBdd('local_otv') as c :
    graph_mmm_filaire = gp.read_postgis('select * from linearisation_bm.mmm_graph_2019', c.connexionPsy)
    graph_mmm_filaire_vertex = gp.read_postgis('select * from linearisation_bm.mmm_graph_2019_vertices_pgr', c.connexionPsy,geom_col='the_geom')

In [10]:
#importer les fichiers finaux
mmm_simple,cle_mmm_rhv=import_fichiers_mmm(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019.shp',
                                            r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\appariementV0.shp')

# Affecter du trafic aux voies non connues

In [13]:
# METHODE A UTILISER
gdf_base=gdf_traf_tot.copy()
lgn_rdpt, dico_noeud, gdf_rhv_rdpt_simple=donnees_tot_rd_pt(gdf_base, 'local_otv_boulot','linearisation_bm', 'graph_rhv_123_v2','graph_rhv_123_v2_vertices_pgr')
gdf_rhv_rdpt_simple5=gdf_rhv_rdpt_simple.copy()

In [ ]:
et.calculer_trafic_3voies(gdf_base,gdf_rhv_rdpt_simple5,lgn_rdpt,graph_filaire_123_vertex)

In [ ]:
gdf_calcul=gdf_base.copy()
gdf_rhv_rdpt_simple2=gdf_rhv_rdpt_simple5.copy()
et.calculer_trafic_mmm(gdf_calcul, gdf_rhv_rdpt_simple2,lgn_rdpt,graph_filaire_123_vertex,mmm_simple,cle_mmm_rhv,dico_noeud)

In [ ]:
#EXPORT RESULTATS
gdf_calcul['id_cpt_exp']=gdf_calcul['id_cpt_2_sens'].fillna('NC')
gdf_calcul['id_cpt_exp']=gdf_calcul.apply(lambda x : ', '.join([str(a) for a in x['id_cpt_exp']]) 
                if isinstance(x['id_cpt_2_sens'],tuple) else str(x['id_cpt_2_sens']), axis=1)
gdf_calcul.reset_index()[['id', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y','source', 'target',
       'idtronc', 'geom','tmjo_2_sens','type_cpt', 'id_cpt_exp']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_123_calcul_mmm_v6.shp')

In [ ]:
# POUR TEST SUR UN NOEUD UNIQUEMNET METHODE 3 VOIES
gdf_base2=gdf_traf_tot.copy()
gdf_rhv_rdpt_simple3=gdf_rhv_rdpt_simple.copy()
#remplacement des sources et targets : 
simplifier_noeud_rdpt(gdf_rhv_rdpt_simple3, dico_noeud)
#puis recalculer le count du nb de ligne par noeud (en otant les lignes qui font les rdpoints)
df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple3)[2:4]
liste_noeud_traite=[]
liste_noeud_mmm=[]
dico_erreur={}
num_noeud=2082
gdf_rhv_rdpt_simple3=verif_index(gdf_rhv_rdpt_simple3,'idtronc')#Îsi idtronc est en index on le repasse dans les colonnes

df_troncon_noeud=et.carac_troncon_noeud(gdf_rhv_rdpt_simple3, gdf_base2, graph_filaire_123_vertex,num_noeud,df_noeuds,lgn_rdpt)
  #si tous les trafics du noeud sont déjà renseignés on passe
  #determiner si on est dans le cas d'une voie pouvant etre estimée par calcul (3 troncon dont 1 seul manquant) ou par analogie avec le MMM
et.maj_trafic_3tronc(df_troncon_noeud)
df_update_traf=df_troncon_noeud[['idtronc','tmjo_2_sens','type_cpt']].set_index('idtronc').drop_duplicates()
df_update_traf.update(pd.DataFrame.from_records(dico_corection_calcul_3_voies, columns=['idtronc','tmjo_2_sens', 'type_cpt']).set_index('idtronc').fillna('NC'))

In [ ]:
#TEST APPARtENANCE A UN ROND POINT
for a in [9314,6404,6403, 8206, 9315] : 
    if a in df_noeuds.noeud.tolist() : 
        print(a)

In [49]:
# POUR TEST SUR UN NOEUD UNIQUEMNET METHODE MMM
gdf_test=gdf_base.copy()
df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
num_noeud=2680
dico_erreur={}
df_troncon_noeud=et.carac_troncon_noeud(gdf_rhv_rdpt_simple5, gdf_test, graph_filaire_123_vertex,num_noeud,df_noeuds,lgn_rdpt)
dico_troncons_noeud=et.creer_dico_troncons_noeud(df_troncon_noeud,gdf_rhv_rdpt_simple5,gdf_base,lgn_rdpt,graph_filaire_123_vertex )
matrice_rhv=et.matrice_troncon_noeud_rhv(df_troncon_noeud,num_noeud,lgn_rdpt,dico_troncons_noeud) 
try :
    joint_fv_mmm_e2=et.estim_mmm_jointure_voies(matrice_rhv,cle_mmm_rhv)
except et.PasCorrespondanceError : 
    dico_erreur[num_noeud]='PasCorrespondanceError'
    df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
    liste_noeud_estim=[n for n in liste_noeud_estim if n not in liste_noeud_traite]
try :
    trafic_inconnus_prior_cat=et.isoler_trafic_inconnu(joint_fv_mmm_e2)
except et.PasDeTraficInconnuError: 
    dico_erreur[num_noeud]='PasDeTraficInconnuError' 
    df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
    liste_noeud_estim=[n for n in liste_noeud_estim if n not in liste_noeud_traite]
try :
    trafc_rens=et.trafic_mmm(gdf_test,trafic_inconnus_prior_cat,mmm_simple,cle_mmm_rhv, df_troncon_noeud,dico_troncons_noeud,lgn_rdpt,dico_noeud,num_noeud)
    df_update_traf=et.calcul_trafic_rhv_depuisMMM(trafc_rens).set_index('idtronc').rename(columns={'tmjo_2_sens_extrapol':'tmjo_2_sens'})
except et.PasDeTraficError as e: 
    dico_erreur[num_noeud]=e 
    df_noeuds,liste_noeud_estim=et.noeuds_estimables(gdf_rhv_rdpt_simple5)[2:4]
    liste_noeud_estim=[n for n in liste_noeud_estim if n not in liste_noeud_traite]

# Voies catégorie 4

## Affectation comptages ponctuels
> Normalement les comptages ponctuels sont deja assignes aux idtronc relatifs au troncon elementaires toutes catégories. Il faut donc isoler les voies non contenues dans le regrouepement des cat 1,2,3

In [176]:
rhv_grp_hors_123,affect_finale_cat4,cpt_perm_cat4=v4.isolerDonneesCat4(gdf_rhv_groupe,gdf_rhv_groupe_123, affect_finale,lgn_proche_perm)
groupe_ident_cpt,cpt_cat4_uniq_sens_dbl,cat4_lgn_pb=v4.jointureTraficLigne(rhv_grp_hors_123,affect_finale_cat4,cpt_perm_cat4)
rhv_grp_hors_123_traf=v4.calculTraficPointComptage(rhv_grp_hors_123,groupe_ident_cpt,cpt_cat4_uniq_sens_dbl,cat4_lgn_pb,[3094, 4687, 8724, 10025])

In [ ]:
#fichier avec cat1,2,3 et comptage sur cat4
gdf_traf_1234=pd.concat([rhv_grp_hors_123_traf.loc[~rhv_grp_hors_123_traf.tmjo_2_sens.isna()],gdf_calcul], axis=0, sort=False)

In [ ]:
gdf_traf_1234.reset_index()[['id', 'ident', 'domanial', 'groupe', 'cat_dig', 'cat_rhv', 'passage',
       'rggraph_nd', 'rggraph_na', 'rgraph_dbl', 'numero', 'cdate', 'mdate',
       'id_ign', 'nature', 'sens', 'codevoie_d', 'importance', 'id_y','source', 'target',
       'idtronc', 'geom','tmjo_2_sens','type_cpt', 'id_cpt_exp']].to_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp')

## Affectation auto à partir du bati pour les habitants
> L'idée c'est d'affecter un tmja a chaque troncon elementaire à partir du nb d'habitantassocier à la ligne la plus proche, puis de moduler ça selon le nb de déplcament motorisée lies au domicile. si apres on arrive aussi a ajouter les tmja pour les voies non reliees aux reseau de cat 1,2,3 c'est parfait.  
***Ce qui manque c'est la prise en compte des sens unique en aller et en retour : il faudrait pouvoir calculer des trajets aller, et des trajets retours***


In [3]:
#import des donnees
gdf_traf_1234, voies_nc, bati,coreesp_bati2=v4.importDonneesBase(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp',
                                                                 r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\voie_non_affectees.shp',
                                                                 r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\bati.shp',
                                                                 r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\ppv_bati-route_V2.csv')

### Recherche des trajets les plus courts vers une voie au trafic connu
> test sur un seul batiment  : 'BATIMENT0000000256067979'.<br> on va alimenter des listes de parcours : list des vertex à tester, list des ident parcouru,list des vertex parcouru  
***LA FONCTION EST ENCORE PERFECTIBLE NOTAMMENT SUR LA GESTION DES IMPASSES DIRECTEMENT COLLEES A DES VERTEX CONNUS***

> **Test : creation d'une classe ensemble de trajet**<br>
>>doit permettre : 
* de verifier le statut de tous les trajets. method : verif_statut_trajets()
* de calculer tous les trajets. method : calcul_trajets()
* trouver les trajets non finis. method : trajets_en_cours()
* creer des nouveaux trajets. method : creer_trajet()
* trouver les lignes a ajouter aux trakets en cours. method : trouver_nouvelles_lignes()
* ajouter une ligne à un trajet existant. method : ajouter_ligne.<br> 

>>Se base sur : 
* lignes depart
* ensemble de lignes
* vertex connus
* vertex_impasse

In [232]:
#recuperer les donnees necessaire a la recherche du trajet le plus rapide vers un point non connu
bati_ligne_proche=v4.definitionVariablesBati(graph_filaire,gdf_traf_1234, bati,coreesp_bati2)
ensemble_lignes,vertex_connus, vertex_impasse=v4.definitionVariablesVoies(graph_filaire,gdf_traf_1234, voies_nc )

In [763]:
%%time
#test sur une ligne
#il peut y avoir des erreurs dans le rhv, comme par exemple l'ident 14844 qui ne touche pas le 24285
ident_proche_bati=str(bati_ligne_proche.loc[bati_ligne_proche['ID']=='BATIMENT0000000257802007'].ident.to_numpy()[0])#risque si ident pas trouver d'IndexError
point_depart=bati_ligne_proche.loc[bati_ligne_proche['ID']=='BATIMENT0000000257802007'].point_proche.values[0]
trajets=v4.ensemble_trajet(ident_proche_bati,point_depart,ensemble_lignes, vertex_connus, vertex_impasse, graph_filaire_vertex)

trajets.initialiser_trajets() 
trajets.allonger_trajet()
trajets.creer_df_tt_trajet()
trajets.filtrer_df_tt_trajet()

trajets.trajet_court=tuple(trajets.df_trajet_cat3_grp_OD.loc[trajets.df_trajet_cat3_grp_OD.longueur_tot==trajets.df_trajet_cat3_grp_OD.longueur_tot.min()].lignes.to_numpy()[0])
trajets.point_arrive=trajets.df_trajet_cat3_grp_OD.loc[trajets.df_trajet_cat3_grp_OD.longueur_tot==trajets.df_trajet_cat3_grp_OD.longueur_tot.min()].points.to_numpy()[0][-1]
#RESULTATS (cf module et attributs)
trajets.trajet_court
#trajets.df_tt_trajet

In [ ]:
#mise en oeuvre si besoin
resultat, dico_df_tt_trajet, liste_bati_erreur=v4.calculTrajets(bati_ligne_proche,ensemble_lignes,vertex_connus, vertex_impasse, graph_filaire_vertex)

In [ ]:
#en cas d'arret pour erreur
#reste_a_faire=export.iloc[:40000].loc[export.iloc[:40000].ID.isin(resultat.bati.tolist())].copy()
index_arret=list_ids.index(resultat.tail(1).bati.values[0])+1
index_bati_erreur=list_ids[index_arret]

In [412]:
#SAVE RESULTATS FINAUX
resultat.to_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_V3.json')

dico_save2={k:v.to_dict() for k, v in dico_df_tt_trajet.items()}
with open(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\df_tt_trajet_resultats_V3.json','w') as f:
    f.write(json.dumps(dico_save2))

In [ ]:
#rappel
with open(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\df_tt_trajet_resultats_V3.json','r') :
    retour_dict=json.loads(f.read())
dico_retour={k:pd.DataFrame.from_dict(v) for k, v in retour_dict.items()}

In [36]:
resultats_finaux=pd.read_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_V3.json')
#ancienne version : r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_tot.json

In [38]:
#ramener les trajets par batiment a une association 1..1 pour le calcul de la somme
resultats_finaux_ordonnees=resultats_finaux.explode('trajet_court')

In [423]:
#si besoin recup des donnees
resultats_finaux_ordonnees=pd.read_json(r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_tot_ordonnes_V3.json')
#jointure avec bati pour recuperer nb personnes et geom
resultats_finaux_ordonnees_pop=bati_ligne_proche[['geom_l','ID','PopT2016']].set_geometry('geom_l').merge(resultats_finaux_ordonnees,left_on='ID', right_on='bati')
#attention des dupliques du à la decoupe du bati par LCSQA
resultats_finaux_ordonnees_pop=resultats_finaux_ordonnees_pop.drop_duplicates(['ID','PopT2016','bati','ident','trajet_court','noeud_cat3'])

In [424]:
#jointire spatiale avec EMC selon secteur pour connaitre le nombre total de deplacement par ident du rhv
zones_emc=gp.read_file(r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\zones\zonage_eda2017.shp')
correspIdentZoneEMD=gp.sjoin(graph_filaire,zones_emc.assign(geomZEmc=zones_emc.geometry),op='intersects')
correspIdentZoneEMD['recou']=correspIdentZoneEMD.apply(lambda x : x['geom'].intersection(x['geomZEmc']).length/x['geom'].length, axis=1)
correspIdentZoneEMDUniq=correspIdentZoneEMD.loc[(correspIdentZoneEMD.recou==correspIdentZoneEMD.groupby('ident').recou.transform(max))][['ident','num_zone']].copy()
emc_nbpers_rhv=resultats_finaux_ordonnees_pop.merge(correspIdentZoneEMDUniq.assign(ident=correspIdentZoneEMDUniq.ident.astype(int)), left_on='trajet_court', right_on='ident')

In [425]:
#recuperer le nombre de trajet moyen par secteur : 
trajMoyen_Emc=pd.read_csv(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\EMC\Linéarisation BdxM et EMC²\trajet_moy_redress_coeq.csv')
#affecter a chaque ident du rhv
trajMoyParLigneRhv=emc_nbpers_rhv.merge(trajMoyen_Emc[['SEC','nb_traj_moy']], left_on='num_zone', right_on='SEC')

In [426]:
#calcul du nombre de deplacements relatif a chaque batiment pour chaque identifiant
trajMoyParLigneRhv['nbDeplacement']=trajMoyParLigneRhv['PopT2016']*trajMoyParLigneRhv['nb_traj_moy']
#estimation du trafic par identifiant du rhv
TraficMoyenEstime=trajMoyParLigneRhv.groupby(['trajet_court','SEC']).agg({'nbDeplacement':sum,'PopT2016':sum,'nb_traj_moy':np.mean}).reset_index()

In [427]:
TraficMoyenEstime.to_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\traficPopulationParIdentRhv_V3.json')

In [428]:
#rappel des resultats : 
TraficMoyenEstime=pd.read_json(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\traficPopulationParIdentRhv_V3.json')

## Affectation auto à partir des points pour les activités
> L'idée c'est d'ajouter au tmja calculé à partir des habitants un nombre de déplacement liés aux activité.<br> Il y a différents types d'activités : les usines ou entreprises qui n'accueillent pas de public, pour lesquelles une estimation du nombre de trajet peut etre fait à partir des effectifs, et les commerces, pour lequels une estimation pourrait etre issu à partir de la catégorie d'ERP.<br> Méthodo : 
1. Filtrer les points en fonction de la ligne du rhv plus proche voisine
1. Analyser le type d'activite : utiliser les codes NAF pour separer les cativites estimable a partir des effectifs, des activites type ERP. Dresser une liste. Filtrer les activites non significative (prof a domicile,...)
1. pour les activites non ERP : analyser celles pour lesquelles il existe une grosse incertitude sur la geoloc des effectifs (i.e les tres grosses ets dont les emplois pourraient etre associe au siege social, les ets dont les employes ne se rendent pas au bureau (BTP par exemple). Pour la partie siege : on ne prend que les effectif de l'établiseement, et on ne prend le siege que si l'attribut quui dit que l'établissement est le siege est coché.
3. pour les activites ERP : proposer une valeur d'accueil du public en fonction de la categorie erp : pas possible, car les ERP de l'IGN ne consernent que la commune de Bordeaux, finalement on opte pour traiter le santé-social et l'enseignement à partir des données de data.gouv

In [4]:
#filtre en fonction du plus proche voisin
activites=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\activites\fi_etabl_p.geojson')
#☼plus proche voisin fait sous postgis
ppvActiviteRhv=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\ppv_activite_rhv.shp')
#isoler les activites le long des voies de categorie 4
activiteRhvCat4=ppvActiviteRhv.loc[ppvActiviteRhv.ident_rhv.isin(gdf_rhv_groupe.loc[gdf_rhv_groupe.cat_rhv.isin(('4','64'))].ident.tolist())].copy()

> ## analyse des codes NAF
ces codes sont fournis par l'INSEE. Au final il est délicat de ne faire le tric que sur les codes, car bcp d'activité peuvent quand même avoir des employés sur site. Le choix est fait de ne supprimer de la liste finale que les entreprises de constructions et certaines de transports, poutr lesquelles les employés sont souvent en déplacement.

In [5]:
#recuperer les donnees de caracterisation des activites
naf5niveaux=pd.read_excel(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\insee\naf2008_5_niveaux.xls')
nafListen5=pd.read_excel(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\insee\naf2008_liste_n5.xls',skiprows=2)

> # ***Si on veut s'epargner toutes les étapes ci dessous, cf la fonction de preparation des données avant passer au calcul des trajets***

In [ ]:
activ_ligne_proche=v4.definitionVariablesActivites(gdf_rhv_groupe,activiteRhvCat4,nafListen5,
                    r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockEtablissement_utf8_Gironde.csv', 
                    r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockUniteLegale_utf8_Gironde.csv',
                    r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\insee\tranchesEffectifs.txt',
                    r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\ppv_finess_rhv_cat4.shp',
                    r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\ppv_enseignement1-2degre_rhv_cat4.shp',
                    r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\education_Nationale\enseignementSecondaire.csv',
                    r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\ppv_enseignementSup_cat4.shp')

>> ### Descrpition des données d'effectif.
A partir des données fournies par BM sur leur [site](https://opendata.bordeaux-metropole.fr/explore/dataset/fi_etabl_p/information) et des données de [data gouv Sirene](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/#_) sur les étblissements, l'idée était de recéupérer les entrerprises et surtout leurs effectifs.  
Le pb c'est que les données sur data gouv soint très parcellaire en terme de nombre d'entreprise avec un effectif renseigné, puisque la base sirene ne decrit que les effectifs si ceux-ci ont été renseigné il y a moins de 2 ans. cf le graph ci-dessous sur les données. le mieux serait d'historiser les donées pour avoir une complétude sur 5 ans par exemple

In [ ]:
#donnees d'effectif (attention que pour les etbalissements recenses par Bordeaux Metropole)
etablissementEnrichi,etablissementEffectifPositifOuvert=v4.etablissementsBdxMet(gdf_rhv_groupe,activiteRhvCat4,nafListen5, 
                                                r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockEtablissement_utf8_Gironde.csv', 
                                                r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockUniteLegale_utf8_Gironde.csv')

In [ ]:
#Visu du nombre d'établissements avec effectifs, non fermés qui ne sont pas des etablissemnet de constrcution ou certins transport
alt.Chart(pd.DataFrame({'type_etablissement':['effectifs connus non null', 'effectif nulls ou inférieur à 1'], 
                        'Nombre_etablissements':[len(etablissementEffectifPositifOuvert),len(etablissementEnrichi)-len(etablissementEffectifPositifOuvert)]}),
         width=400, title='Répartition des données d\'activité 2020 selon la disponibilité de l\'effectif').mark_bar().encode(x=alt.X('type_etablissement',title='Type de données dispo', axis=alt.Axis(orient='bottom',labelAngle=0)),
                                      y=alt.Y('Nombre_etablissements',title='Nombre d\'établissements'))

>> ### Autre données d'établissement : Finess
les données [Finess](https://www.data.gouv.fr/fr/datasets/finess-extraction-du-fichier-des-etablissements/) ont été testées. L'idée est de récupérer ici les établissement non recensées dans les activités de BdxMet, et qui sont ouvertes et ont un effectifs selon les données Sirene

In [ ]:
finessSirene=v4.fichierSireneFiness(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockEtablissement_utf8_Gironde.csv',
                                    r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\ppv_finess_rhv_cat4.shp',
                                    etablissementEffectifPositifOuvert,
                                   r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockUniteLegale_utf8_Gironde.csv')

>> ### Autre données d'établissement : Enseignement
Sur data gouv on trouve pas mal de données, qui ne sont pas dans les données d'activités de BdxMet (verif visu sous Qgis pour 1er et 2nd degré, par croisement du numero siret pour enseignement supérieur) : 
>- les données [d'enseignement geolocalisées](https://www.data.gouv.fr/fr/datasets/adresse-et-geolocalisation-des-etablissements-denseignement-du-premier-et-second-degres-1/) pour le primaire et secondaire,
>- pour le primaire on va récupérer la données depuis sirene grace a une correspondance de pattern sur l'adresse
>- les données sur [l'enseignement secondaire](https://www.data.gouv.fr/fr/datasets/ideo-structures-denseignement-secondaire/) en lien avec les numeros siren vont remplacer les données geolocalisées sus-citées
>- les donnees sur l'enseignement superieur : [en lien avec un numero siren](https://www.data.gouv.fr/fr/datasets/ideo-structures-denseignement-superieur/) ou en [effectif etudiants inscrit à la commune](https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits/information/?disjunctive.rgp_formations_ou_etablissements)<br>
L'idée est de travailler uniquement avec les données de l'enseignement supérieurs liées aux codes siren, car les part modales des effectifs etudiants sont incertaines

In [ ]:
enseignementSirene=v4.fichierEnseignementSirene(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\ppv_enseignement1-2degre_rhv_cat4.shp',
                                  r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockEtablissement_utf8_Gironde.csv',
                                  r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\StockUniteLegale_utf8_Gironde.csv',
                                 r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\education_Nationale\enseignementSecondaire.csv',
                                  r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\ppv_enseignementSup_cat4.shp',
                                               etablissementEffectifPositifOuvert)[0]

> ## **Affectation aux troncons routier**
on va reprendre le même processus que pour les batiments, il va juste falloir adapter les variables d'entrée : 
- le bati est remplacé par la table effectifEtablissement
- la table de correspondance est ppvActiviteRhv (fichier shape) avec id de l'activite = ident et id de la ligne = ident_rhv
- les données de voies a renseigner sont les mm que pour le bati, isues de v4.definitionVariablesVoies

In [ ]:
#mise en ouevre si besoin
resultat, dico_df_tt_trajet, liste_bati_erreur=v4.calculTrajets(activ_ligne_proche,ensemble_lignes,vertex_connus, vertex_impasse, graph_filaire_vertex)

In [426]:
#save resultats
resultat.to_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\resultat_trajets_base.json')

In [305]:
#rappel resultats intermediaires
resultatActivite=pd.read_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\resultat_trajets_base.json')

In [306]:
#qq modif / nettoyage
resultatActivite.dropna(inplace=True)
resultatActivite['trajet_court']=resultatActivite.trajet_court.apply(lambda x : tuple(x))
resultatActivite.drop_duplicates(['bati','ident','noeud_proche','trajet_court','noeud_cat3'],inplace=True)
#recuperation des donnees d'effectifs
resultatActivitePaLigneEffectifs=resultatActivite.explode('trajet_court').merge(activ_ligne_proche[['ID','effectifMoyen','geom_p']], left_on='bati', right_on='ID')

In [307]:
#jointire spatiale avec EMC selon secteur pour connaitre le nombre total de deplacement par ident du rhv
zones_emc=gp.read_file(r'D:\Boulot\Lin_bm\C19SA0101\Doc_travail\Donnees_produites\Donnees\zones\zonage_eda2017.shp')
correspIdentZoneEMD=gp.sjoin(graph_filaire,zones_emc.assign(geomZEmc=zones_emc.geometry),op='intersects')
correspIdentZoneEMD['recou']=correspIdentZoneEMD.apply(lambda x : x['geom'].intersection(x['geomZEmc']).length/x['geom'].length, axis=1)
correspIdentZoneEMDUniq=correspIdentZoneEMD.loc[(correspIdentZoneEMD.recou==correspIdentZoneEMD.groupby('ident').recou.transform(max))][['ident','num_zone']].copy()
ActiviteEffectifRhv=resultatActivitePaLigneEffectifs.merge(correspIdentZoneEMDUniq, left_on='trajet_court', right_on='ident').drop('ident_y', axis=1).rename(columns={'ident_x':'ident'})

In [313]:
#recuperer les donnees EMC
trajetActiviteMOyen=pd.read_csv(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\moyenneDeplacementTravail.csv')
#affecter a chaque ident du rhv
trajMoyParLigneRhv=ActiviteEffectifRhv.merge(trajetActiviteMOyen[['SEC_emploi','nb_moy_trajet_pers','ratio_trajet_motorise']], left_on='num_zone', right_on='SEC_emploi')
trajMoyParLigneRhv['nbDeplacementActivite']=trajMoyParLigneRhv.effectifMoyen*trajMoyParLigneRhv.nb_moy_trajet_pers*trajMoyParLigneRhv.ratio_trajet_motorise
#estimation du trafic par identifiant du rhv
TraficMoyenActiviteEstime=trajMoyParLigneRhv.groupby(['trajet_court','SEC_emploi']).agg({'nbDeplacementActivite':sum,'effectifMoyen':sum,'nb_moy_trajet_pers':np.mean,'ratio_trajet_motorise':np.mean}).reset_index()
#estimation des trafic lies a l'activite par ident du rhv 
TraficMoyenActiviteEstime.to_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\traficParIdentRhv.json')

## Fusion des sources de données estimées de population et d'activites

In [7]:
#charger les sources de données
TraficMoyenActiviteEstime=pd.read_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\activite\traficParIdentRhv.json')
TraficMoyenPopulationEstime=pd.read_json(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\traficPopulationParIdentRhv_V3.json')
#Fusion et mise en forme
TraficActiviteUniquement=TraficMoyenActiviteEstime.loc[~TraficMoyenActiviteEstime.trajet_court.isin(TraficMoyenPopulationEstime.trajet_court.tolist())]
TraficParIdent=pd.concat([TraficMoyenPopulationEstime.merge(TraficMoyenActiviteEstime.loc[TraficMoyenActiviteEstime.trajet_court.isin(TraficMoyenPopulationEstime.trajet_court.tolist())], on='trajet_court', how='left'),
           TraficActiviteUniquement.rename(columns={'SEC_emploi':'SEC'})],axis=0)[['trajet_court','SEC','nbDeplacement','PopT2016','nb_traj_moy','nbDeplacementActivite',
                                                                                 'effectifMoyen','nb_moy_trajet_pers','ratio_trajet_motorise']].rename(columns={'nbDeplacement':'nbDeplacementPop',
                                                                                 'nb_traj_moy':'nb_traj_moyPop'}).fillna(0)
TraficParIdent['tmja']=TraficParIdent.nbDeplacementPop+TraficParIdent.nbDeplacementActivite

## Analyse des données
L'idée est de regarder comment se comporte les calculs de tmja sur le svoies de catégories 4 et 64, par rapport aux comptages qui ont pu etre réalisés par BdxMet sur ces voies.  
Les voies sont régroupées par zones EMD, afin de disposer de suffsamment de points par zone pour avoir une analyse.   
En premiere lecture ont se rend compte qu'une majorité de zone (1,2,3,4,6,7,8,9,10,12,13,18,19) ont entre 60 et 90 % de leur voies pour lesquelles le trafic calculé ne représente que pas plus de 20% du trafic mesuré. Les voies calculées représentant entre 60 et 100% du trafic mesuré représente rarement plus de 20% de l'ensemble des voies calculées.  
En regroupant le svoies par cat_dig, on se rend compte dans un premier temps qu'une infime partie des voies à estimer est classée en catégorie 3, c'est à dire voie urbaine structurante, ce qui n'est pas logique avec le classement en cat_rhv 4 ou 64 i.e voie de déssert locale. on va donc limiter les traitements suivants sur la base des données des voies mesurées en cat_dig = 4 ou 5
Dans un second temps ont divise les trafics calculés en quartile, puis on analyse les ratio entre le trafic calculé et mesuré selon ces quartiles. le résultat est très homogène, et si certains secteurs et quartile peuvent faire l'objets de ratio plutot homogène, d'autre sont très hétéroclite. Le même raisonement tenté en prenant des quartiles de valuers mesurées aboutit au même résultat.

L'objectif est donc de déterminer, ppour chaque secteur EMD, la valeur moyenne de correctif à appliquer en fonction du tmja calculé, et d'y associer un indicateur bref de confiance : faible, moyen, bon. 
En parrallèle on tente une approche par regression, en utilisant un algorythme de support vector regression nourris par les 450 points que l'on connait, car tous les algorithmes de prédiction linéaire testés donnent des résultats avec une mauvais affiliation sur les points déjà connu (r²<0.25). Il faudrait vraiment qu'un spécialiste se penche sur la question, mais la disparités des rapports entre le tmja mesure et calcule est vraiment grande. Cela peut provenir des multiples sources d'erreure : affecation de la population au bati, calcul des trajets vers les cat_rhv 1,2,3 non représentatif, faible prise en compte des activités, car pas de données d'effectif connu, pas de prise en compte du trafic de passage ou de destination non recensée (fréquentation des ERP).

Au final le resultat nous fait perdre la corrélation entre les sommes de population par trajet obtenus par les calculs de parcours, mais il semble permettre d'obtenir des données représentative, avec un SVR poly prenant en entrée le tma calculé, la population et les effectifs.

In [8]:
#ensuite on va cherccher a comparer avec les donnees de comptage 
traf_cat_4=gdf_traf_1234.loc[gdf_traf_1234.cat_rhv.isin(('4','64'))][['ident','tmjo_2_sen','id_cpt_exp','geometry']].copy()
compTraficMesureEstime=traf_cat_4.assign(ident=traf_cat_4.ident.astype(int)).merge(TraficParIdent.rename(columns={'trajet_court':'ident'}),on='ident')
compTraficMesureEstimeTot=traf_cat_4.assign(ident=traf_cat_4.ident.astype(int)).merge(TraficParIdent.rename(columns={'trajet_court':'ident'}),on='ident', how='right')
test_stats=compTraficMesureEstime.assign(mjMes_calc=compTraficMesureEstime.tmja/compTraficMesureEstime.tmjo_2_sen).merge(graph_filaire[['ident', 'cat_dig']].assign(ident=graph_filaire.ident.astype(int)), on='ident')
#mais on va se limiter sur les lignes de cat_dig = 4 ou 5

In [9]:
#affectation des quantiles selon le ratio entre mesure et calcule, filtre dans une df dediee
labels_qtl=[f'quantile {a}' for a in range(5,105,5)]
statsAffiliationTrafic=test_stats.merge(pd.qcut(test_stats.mjMes_calc,20,labels_qtl).cat.as_ordered(), left_index=True,right_index=True).rename(
    columns={'mjMes_calc_x':'mjMes_calc','mjMes_calc_y':'quantile'})
statsAffiliationTraficFiltreQuantile=statsAffiliationTrafic.loc[~statsAffiliationTrafic['quantile'].isin(['quantile 100','quantile 5'])]

In [10]:
#visu de répartition des extimation de trafic en fonction de l'écart avec la réalité, par zone EMD
labels=['0-20%','20-40%', '40-60%', '60-80%','80-100%','100-120%', '120-140%','140-160%','160-200%','sup200%']
listGrpZone=[]
for s in statsAffiliationTraficFiltreQuantile.SEC.unique() : 
    grpZone=pd.cut(statsAffiliationTraficFiltreQuantile.loc[statsAffiliationTraficFiltreQuantile.SEC==s].mjMes_calc, 
            bins=[0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,2,100], labels=labels).rename_axis('nb_ligne').reset_index().groupby('mjMes_calc').count().reset_index()
    grpZone['zoneEmd']=s
    listGrpZone.append(grpZone)
donneesFiltree=pd.concat(listGrpZone, axis=0)

barNormalisees=alt.Chart(donneesFiltree.reset_index(), width=600, height=300).mark_bar().encode(x=alt.X('zoneEmd:N',title='Zone EDA'),
                                            y=alt.Y('nb_ligne:Q',stack="normalize", 
                                                    axis=alt.Axis(format='%', title=['Pourcentage de ligne par classe', 'de rapport Mesure/calcul']),
                                                   sort=labels),
                                            color=alt.Color('mjMes_calc', sort=labels),
                                           order = alt.Order('index'))
barAbsolues=alt.Chart(donneesFiltree.reset_index(), width=600, height=300).mark_bar().encode(x=alt.X('zoneEmd:N',title='Zone EDA'),
                                            y=alt.Y('nb_ligne:Q',sort=labels,axis=alt.Axis(title=['Nombre de ligne par classe','de rapport calcul / mesure'])),
                                            color=alt.Color('mjMes_calc', sort=labels, title=['part du TMJA calculé', 'par rapport au TMJA mesuré']),
                                           order = alt.Order('index'))
graphRapportCalcMesureZoneEmd=alt.vconcat(barNormalisees,barAbsolues)

In [11]:
graphRapportCalcMesureZoneEmd

alt.VConcatChart(...)

In [13]:
#mm raisonnement, mais par cat_dig
labels=['0-20%','20-40%', '40-60%', '60-80%','80-100%','100-120%', '120-140%','140-160%','160-200%','sup200%']
listGrpZone=[]
for s in statsAffiliationTraficFiltreQuantile.cat_dig.unique() : 
    grpZone=pd.cut(statsAffiliationTraficFiltreQuantile.loc[statsAffiliationTraficFiltreQuantile.cat_dig==s].mjMes_calc, 
            bins=[0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,2,100], labels=labels).rename_axis('nb_ligne').reset_index().groupby('mjMes_calc').count().reset_index()
    grpZone['cat_dig']=s
    listGrpZone.append(grpZone)
donneesFiltree=pd.concat(listGrpZone, axis=0)

barNormalisees=alt.Chart(donneesFiltree.reset_index(), width=500, height=600).mark_bar().encode(x='cat_dig:N',
                                            y=alt.Y('nb_ligne:Q',stack="normalize", 
                                                    axis=alt.Axis(format='%', title='Pourcentage de ligne par classe de rapport Mesure/calcul'),
                                                   sort=labels),
                                            color=alt.Color('mjMes_calc', sort=labels),
                                           order = alt.Order('index'))
barAbsolues=alt.Chart(donneesFiltree.reset_index(), width=500, height=600).mark_bar().encode(x='cat_dig:N',
                                            y=alt.Y('nb_ligne:Q',sort=labels,axis=alt.Axis(title='Nombre de ligne par classe de rapport Mesure/calcul')),
                                            color=alt.Color('mjMes_calc', sort=labels),
                                           order = alt.Order('index'))
graphRapportCalcMesureCatdig=barNormalisees | barAbsolues

In [14]:
graphRapportCalcMesureCatdig

alt.HConcatChart(...)

In [12]:
#preparation des donnees
listZoneStatDetailsCalcule=[]
listZoneStatDescribeCalcule=[]
listZoneStatDetailsMesure=[]
listZoneStatDescribeMesure=[]
DicoChartCatDig={}
for s in statsAffiliationTraficFiltreQuantile.SEC.unique() : 
    #analyse des classes de ratio par classes de trafic calculé
    zoneEmd=statsAffiliationTraficFiltreQuantile.loc[((statsAffiliationTraficFiltreQuantile.SEC==s)) & (statsAffiliationTraficFiltreQuantile.cat_dig.isin(('4','5')))].copy()
    #repartition selon le trafic calculé
    zoneEmdStatsCalcule=zoneEmd.merge(pd.qcut(zoneEmd.tmja,4).apply(lambda x : f'{round(x.left)} - {round(x.right)}'), left_index=True, right_index=True).rename(
        columns={'tmja_y':'quartile_tmja_zone'}).sort_values('quartile_tmja_zone')
    zoneEmdDescribeCalcule=zoneEmdStatsCalcule.groupby('quartile_tmja_zone').describe().loc[:,['mjMes_calc', 'tmjo_2_sen']].reset_index()
    listZoneStatDetailsCalcule.append(zoneEmdStatsCalcule)
    listZoneStatDescribeCalcule.append(zoneEmdDescribeCalcule.assign(SEC=s))
ZoneStatDetailsCalcule=pd.concat(listZoneStatDetailsCalcule,axis=0)
ZoneStatDescribeCalcule=pd.concat(listZoneStatDescribeCalcule,axis=0)

In [13]:
ZoneStatDescribeCalcule=ZoneStatDescribeCalcule.loc[:,['quartile_tmja_zone','mjMes_calc','tmjo_2_sen','SEC']].T.reset_index(drop=True).T

In [14]:
ZoneStatDescribeCalcule.columns=['quartile_tmja_zone',
                                 'count','RapCalcMesmean','std','RapCalcMesMin','25%','50%','75%','RapCalcMesMax',
                                 'MjMesucount','MjMesumean','MjMesustd','MjMesuMin','MjMesu25%','MjMesumediane','MjMesu75%','MjMesuMax',
                                 'SEC']
ZoneStatDescribeCalcule=ZoneStatDescribeCalcule[['quartile_tmja_zone','count','MjMesuMin','MjMesuMax','std','25%','50%','75%','SEC']]

In [87]:
#visu des repartition

#affecation des donnees statistiques à chaque point
tmjaMinMax=ZoneStatDescribeCalcule.copy()
tmjaMinMax['num_quartil']=tmjaMinMax.index
tmjaMinMax.reset_index(drop=True, inplace=True)
tmjaMinMax=tmjaMinMax.assign(tmja_min=lambda x :x.quartile_tmja_zone.apply(lambda x : int(x.split(' - ')[0])),tmja_max=lambda x :x.quartile_tmja_zone.apply(lambda x : int(x.split(' - ')[1])))
tousIndic=statsAffiliationTraficFiltreQuantile.sort_values(['SEC','tmja']).merge(tmjaMinMax, on='SEC')
tousIndic=tousIndic.loc[(round(tousIndic.tmja)>=tousIndic.tmja_min) & (tousIndic.tmja<tousIndic.tmja_max)]
#creation du graph pour chaque zone emd
dicoGraphQuartile={}
for s in statsAffiliationTraficFiltreQuantile.SEC.unique() : 
    dfZone=tousIndic.loc[tousIndic.SEC==s].copy()
    legendValues=dfZone.quartile_tmja_zone.unique()
    scale = alt.Scale(domain=legendValues,
                  range=['red', 'blue', 'green', 'orange'])
    base=alt.Chart(dfZone, title=f'zone EDA {s}',width=250, height=200)
    charteCompMesureCalcul=base.mark_line(color='black').encode(x=alt.X('tmja', title='trafic calcule'), 
                                                                y=alt.Y('tmja',title='Trafic Mesure'))+base.mark_point().encode(
                                                                x='tmja', 
                                                                y=alt.Y('tmjo_2_sen',title='Trafic Mesure'), color=alt.Color('quartile_tmja_zone', scale=scale ))
    CharteRapportMesureCalcul=base.mark_point().encode(x=alt.X('tmja', title='trafic calcule'), 
                                                       y=alt.Y('mjMes_calc', title='tmja calcule / tmja mesure'), 
                                                       color=alt.Color('quartile_tmja_zone:N',scale=scale))+ base.mark_line().encode(
                                                       x=alt.X('tmja', title='trafic calcule'), 
                                                       y=alt.Y('50%', title='tmja calcule / tmja mesure'), 
                                                       color=alt.Color('quartile_tmja_zone:N',scale=scale))+base.mark_area(opacity=0.3).encode(
                                                       x='tmja',y='25%',y2='75%', color=alt.Color('quartile_tmja_zone:N',title=['Quartile de','TMJA calculé'],scale=scale,))
    dicoGraphQuartile[s]=(CharteRapportMesureCalcul | charteCompMesureCalcul)
GraphVisuQuartile=alt.vconcat(*[dicoGraphQuartile[i] for i in range(15,20)]).resolve_scale(color='independent')

In [88]:
GraphVisuQuartile

alt.VConcatChart(...)

In [63]:
#visualisation des ratio entre TMJA calculé et le TMJA mesuré, avec ecarttype, par quartile de tmja_calculé
dicoGraphTmjaCalcule={}
for s in statsAffiliationTraficFiltreQuantile.SEC.unique() : 
    source=ZoneStatDescribeCalcule.loc[ZoneStatDescribeCalcule.SEC==s]
    # the base chart
    base = alt.Chart(source, width=200, title=f"zone EMD {s} ; Nb de lignes par quartile = {source['count'].max()}").transform_calculate(
        ymin="datum['50%']-datum.std",
        ymax="datum['50%']+datum.std")
    # generate the points
    pointsMid = base.mark_point(
        filled=True,size=90,color='red').encode(
        x=alt.X('quartile_tmja_zone', sort=source.quartile_tmja_zone.tolist(), axis=alt.Axis(title='quartiles de valeurs de tmja calculé')),
        y=alt.Y('50%', scale=alt.Scale(domain=(-0.5, 1.5)),axis=alt.Axis(title='ratio tmja calculé / tmja mesuré',format='%')),
        tooltip='50%')
    pointsHigh = base.mark_point(
        filled=True,size=90,color='blue').encode(
        x=alt.X('quartile_tmja_zone', sort=source.quartile_tmja_zone.tolist(), axis=alt.Axis(title='quartiles de valeurs de tmja calculé')),
        y=alt.Y('75%', scale=alt.Scale(domain=(-0.5, 1.5)),axis=alt.Axis(title='ratio tmja calculé / tmja mesuré',format='%')))
    pointsLow = base.mark_point(
        filled=True,size=90,color='green').encode(
        x=alt.X('quartile_tmja_zone', sort=source.quartile_tmja_zone.tolist(), axis=alt.Axis(title='quartiles de valeurs de tmja calculé')),
        y=alt.Y('25%', scale=alt.Scale(domain=(-0.5, 1.5)),axis=alt.Axis(title='ratio tmja calculé / tmja mesuré',format='%')))
    #creation d'une visu des points sources
    pointsSources=alt.Chart(ZoneStatDetailsCalcule.loc[ZoneStatDetailsCalcule.SEC==s], width=200).mark_circle(
        filled=True,size=35,color='gray').encode(
        x=alt.X('quartile_tmja_zone', sort=source.quartile_tmja_zone.tolist(), axis=alt.Axis(title='quartiles de valeurs de tmja calculé')),
        y=alt.Y('tmjo_2_sen',axis=alt.Axis(title='Quantité de véhicules mesurée',titleColor='gray')),
        tooltip='tmjo_2_sen')
    # generate the error bars
    errorbars = base.mark_errorbar().encode(
        x=alt.X('quartile_tmja_zone', sort=source.quartile_tmja_zone.tolist(),axis=alt.Axis(title='quartiles de valeurs de tmja calculé')),
        y=alt.Y("ymin:Q",axis=alt.Axis(title='ratio tmja calculé / tmja mesuré',format='%')),y2="ymax:Q")

    dicoGraphTmjaCalcule[s]= alt.layer((pointsMid + pointsHigh + pointsLow + errorbars), pointsSources).resolve_scale(y='independent')
VisuQualiteEstimationCalcul=alt.vconcat(*[alt.hconcat(*[dicoGraphTmjaCalcule[i+j] for i in range(1,5) if i+j<20]) for j in range(0,19,4)])

In [64]:
VisuQualiteEstimationCalcul

alt.VConcatChart(...)

## Calcul des niveaux de trafic estimes : Méthode
attention, on peu charger directement les resultats en passant au paragraphe suivant, ici c'est la méthodo

In [65]:
#par analyse perso
tmjaMinMaxAnalyse=pd.concat([tmjaMinMax,pd.read_csv(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\type_ecart_type.csv')], axis=1)

In [19]:
#affecter un pourcentage de recalage manuel: 
tmjaMinMaxAnalyse['MesCal']=tmjaMinMaxAnalyse.apply(lambda x : x['50%'] if x['qtl_retenu']=='mediane' else x['75%'], axis=1)
#affecter ces pourcentages à l'ensemble des voies non connues
def affecterPct(tmja, tmja_min, tmja_max,num_quartil ) : 
    """
    pour affecter le pourcentage calculer à partir des voies connues, il faut que l'o puisse 
    aller chercher les trafics sous la valeur basse du premier quartil et au dessus de la valeur haute du dernier
    """
    if num_quartil==0 : 
        if tmja<tmja_max : 
            return True
        else : return False
    elif num_quartil==3 : 
        if tmja>=tmja_min : 
            return True
        else : return  False
    else : 
        if tmja_min<=tmja<tmja_max : 
            return True
        else : return False
    
RegroupeTrafparIdent=compTraficMesureEstimeTot.merge(tmjaMinMaxAnalyse[['SEC','tmja_min','tmja_max','MesCal','num_quartil']],on='SEC', how='left').drop('geometry', axis=1)
RegroupeTrafparIdentFiltre=RegroupeTrafparIdent.loc[RegroupeTrafparIdent.apply(lambda x : affecterPct(x['tmja'], x['tmja_min'], x['tmja_max'],x['num_quartil']), axis=1) ].copy()
#affecter du trafic : 
RegroupeTrafparIdentFiltre['tmja_estime']=RegroupeTrafparIdentFiltre.tmja*(1/RegroupeTrafparIdentFiltre.MesCal)

In [ ]:
#test avec machien learniing
#apres plusieurs test on part sur du svr en poly, ce serait bien de remettre un test ou deux en lineaire
#prepa des données
dicoParamIa={'1param':{'X_test':[[a] for a in RegroupeTrafparIdentFiltre.loc[(RegroupeTrafparIdentFiltre.SEC==s)].tmja.to_numpy()],
                       'X_train':[[a] for a in RegroupeTrafparIdentFiltre.loc[~RegroupeTrafparIdentFiltre.tmjo_2_sen.isna() & (RegroupeTrafparIdentFiltre.SEC==s)].tmja.to_numpy()]},
            '3param':{'X_test':[[r.tmja, r.PopT2016, r.effectifMoyen] for r in RegroupeTrafparIdentFiltre.loc[RegroupeTrafparIdentFiltre.SEC==s].itertuples()],
                       'X_train':[[r.tmja, r.PopT2016, r.effectifMoyen] for r in RegroupeTrafparIdentFiltre.loc[~RegroupeTrafparIdentFiltre.tmjo_2_sen.isna() & (RegroupeTrafparIdentFiltre.SEC==s)].itertuples()]}}
dicoEstimIa={}
qt = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
#pour chaque scteuer
for s in RegroupeTrafparIdentFiltre.SEC.unique() : 
    RegroupeTrafparIdentFiltre['rapMesCalc']=RegroupeTrafparIdentFiltre.tmja/RegroupeTrafparIdentFiltre.tmjo_2_sen #calcul du rapport de base
    y=[[a] for a in RegroupeTrafparIdentFiltre.loc[~RegroupeTrafparIdentFiltre.tmjo_2_sen.isna() & (RegroupeTrafparIdentFiltre.SEC==s)].rapMesCalc.to_numpy()]
    dicoParamIa={'1param':{'X_test':[[a] for a in RegroupeTrafparIdentFiltre.loc[(RegroupeTrafparIdentFiltre.SEC==s)].tmja.to_numpy()],
                       'X_train':[[a] for a in RegroupeTrafparIdentFiltre.loc[~RegroupeTrafparIdentFiltre.tmjo_2_sen.isna() & (RegroupeTrafparIdentFiltre.SEC==s)].tmja.to_numpy()]},
            '3param':{'X_test':[[r.tmja, r.PopT2016, r.effectifMoyen] for r in RegroupeTrafparIdentFiltre.loc[RegroupeTrafparIdentFiltre.SEC==s].itertuples()],
                       'X_train':[[r.tmja, r.PopT2016, r.effectifMoyen] for r in RegroupeTrafparIdentFiltre.loc[~RegroupeTrafparIdentFiltre.tmjo_2_sen.isna() & 
                                                                                                              (RegroupeTrafparIdentFiltre.SEC==s)].itertuples()]}}
    for k, v in dicoParamIa.items() : #pour cahque possibilité de jeu de parametre
        X_train_scaled_qt=qt.fit_transform(preprocessing.scale(v['X_train']))
        X_test_scaled_qt=qt.fit_transform(preprocessing.scale(v['X_test']))
        svr_poly= SVR(kernel='poly', C=1 )
        svr_poly.fit(X_train_scaled_qt, [a[0] for a in y])
        RegroupeTrafparIdentFiltre.loc[(RegroupeTrafparIdentFiltre.SEC==s),f'MesCalIA{k[:2]}']=svr_poly.predict(X_test_scaled_qt)
#ATTENTION, pour certaines lignes (environ 1000, là ou un des deux nombre de deplacement est tres bas) les coeff sont très très bas avec les 3 parametres, don con prend ceux du 1parametres
ResultatsEstim.loc[ResultatsEstim['MesCalIA3p']<0.01,'MesCalIA3p']=ResultatsEstim.loc[ResultatsEstim['MesCalIA3p']<0.01].MesCalIA1p
#par ailleur sur le secteur 8 il semble que par rapport au reste il y ai des différences fortes, donc au final la meileure solution semble le type 1 parametres, à corriger sur ses valeusr les plus fortes

In [22]:
#mettre en forme et separer les sources de données
ResultatsEstim=RegroupeTrafparIdentFiltre.drop(['id_cpt_exp','tmja_min','tmja_max'], axis=1)
ResultatsEstim['estm_IA_1p']=ResultatsEstim.apply(lambda x : x.tmja*(1/x.MesCalIA1p) if x.MesCalIA1p>0 else 0, axis=1)
ResultatsEstim['estm_IA_3p']=ResultatsEstim.apply(lambda x : x.tmja*(1/x.MesCalIA3p) if x.MesCalIA3p>0 else 0, axis=1)
ResultatsEstimGeom=graph_filaire[['ident','geom','domanial','cat_dig','cat_rhv','groupe','numero','passage','rggraph_nd','rggraph_na','rgraph_dbl']].assign(ident=graph_filaire.ident.astype(int)).merge(ResultatsEstim, on ='ident')
ResultatsEstimGeom['longueur']=ResultatsEstimGeom.geometry.length
ResultatsEstimGeom=ResultatsEstimGeom.assign(veh_km_estime=ResultatsEstimGeom.longueur/1000*ResultatsEstimGeom.tmja_estime,
                         veh_km_IA_1p=ResultatsEstimGeom.longueur/1000*ResultatsEstimGeom.estm_IA_1p,
                         veh_km_IA_3p=ResultatsEstimGeom.longueur/1000*ResultatsEstimGeom.estm_IA_3p)

## Calcul des niveaux de trafic estimes : Visu et chargement resultats
Charger directement les resultats

In [39]:
ResultatsEstimGeom=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\affectationComplete_VTravail.shp')
ResultatsEstimGeom.rename(columns={'tmja_estim':'tmja_estime','veh_km_est':'veh_km_estime', 'veh_km_IA_':'veh_km_IA_1p', 'veh_km_I_1':'veh_km_IA_3p',
                                  'nbDeplacem':'nbDeplacementPop','nbDeplac_1':'nbDeplacementActivite'}, inplace=True)
ResultatsEstimGeom['longueur']=ResultatsEstimGeom.longueur/1000
ResultatsEstimGeomCptConnu=ResultatsEstimGeom.loc[~ResultatsEstimGeom.tmjo_2_sen.isna()].copy()
ResultatsEstimGeomCptInconnu=ResultatsEstimGeom.loc[ResultatsEstimGeom.tmjo_2_sen.isna()].copy()

In [18]:
#comparaison des estimations sur les données connues : on penche plutot sur le poly 1 params, à modifier par la valeur estimee sur ses valuers les plus fortes, cf plus bas
ResultatsEstimCpt=ResultatsEstimGeomCptConnu.set_index(['tmjo_2_sen','SEC'])[['tmja_estime','estm_IA_1p','estm_IA_3p']].stack().reset_index()
ResultatsEstimCpt.columns=['tmjo_2_sen','SEC','type_estim','tmjo_estim']
dicoCompEstime={}
for s in ResultatsEstimCpt.SEC.unique() : 
    dfZone=ResultatsEstimCpt.loc[ResultatsEstimCpt.SEC==s].copy()
    points_estim=alt.Chart(dfZone,title=f'zone EDA {s}', width=500).mark_point().encode(x='tmjo_estim', y='tmjo_2_sen', color=alt.Color('type_estim', title=['source de','l\'estimation']), 
                                                                                        shape=alt.Shape('tmjo_2_sen:O', legend=None))
    ligne=alt.Chart(dfZone).mark_line(color='black').encode(x=alt.X('tmjo_2_sen', title='trafic calcule'), y=alt.Y('tmjo_2_sen',title='Trafic Mesure'))
    dicoCompEstime[s]=(points_estim+ligne).interactive()
GraphCompEstime=alt.vconcat(*[alt.hconcat(*[dicoCompEstime[str(i+j)] for i in range(1,4) if i+j<20]) for j in range(0,19,3)])

In [19]:
dicoCompEstime['16'].properties(width=500)

alt.LayerChart(...)

In [20]:
GraphCompEstime

alt.VConcatChart(...)

In [21]:
#analyse des resultats par classe de trafic en veh.km
labels=['0 - 500','500 - 1000', '1000 - 2000', '2000 - 4000', '4000 - 8000','sup 8000']
ResultatsEstimGeomCptInconnu['interval_IA_3p']=pd.cut(ResultatsEstimGeomCptInconnu.estm_IA_3p,[0,500,1000,2000,4000,8000,100000], labels=labels)
ResultatsEstimGeomCptInconnu['interval_IA_1p']=pd.cut(ResultatsEstimGeomCptInconnu.estm_IA_1p,[-0,500,1000,2000,4000,8000,100000], labels=labels)
ResultatsEstimGeomCptInconnu['interval_estime']=pd.cut(ResultatsEstimGeomCptInconnu.tmja_estime,[0,500,1000,2000,4000,8000,100000], labels=labels)
ResultatsEstimGeomCptConnu['interval_tmjo']=pd.cut(ResultatsEstimGeomCptConnu.tmjo_2_sen,[0,500,1000,2000,4000,8000,100000], labels=labels)
ResultatsEstimGeomCptInconnuGrpVehKm=ResultatsEstimGeomCptInconnu[['veh_km_estime','veh_km_IA_1p','veh_km_IA_3p']].stack().reset_index()
ResultatsEstimGeomCptInconnuGrpVehKm.columns=['ident', 'type_estim','veh_km']
graphRepartitionTmja=(alt.Chart(ResultatsEstimGeomCptInconnu.groupby('interval_IA_3p').longueur.sum().reset_index(),title='lignes à trafic inconnu').mark_bar().encode(x=alt.X('interval_IA_3p',title='estim_IA_3p',sort=labels) , y=alt.Y('longueur', title='Longueur (km)')) |
 alt.Chart(ResultatsEstimGeomCptInconnu.groupby('interval_IA_1p').longueur.sum().reset_index(),title='lignes à trafic inconnu').mark_bar().encode(x=alt.X('interval_IA_1p',sort=labels, title='estim_IA_1p'), y=alt.Y('longueur', title='Longueur (km)')) | 
 alt.Chart(ResultatsEstimGeomCptInconnu.groupby('interval_estime').longueur.sum().reset_index(), title='lignes à trafic inconnu').mark_bar().encode(x=alt.X('interval_estime',sort=labels,title='tmja_estim'), y=alt.Y('longueur', title='Longueur (km)')) | 
alt.Chart(ResultatsEstimGeomCptConnu.groupby('interval_tmjo').longueur.sum().reset_index(), title='lignes à trafic connu').mark_bar().encode(x=alt.X('interval_tmjo',sort=labels), y=alt.Y('longueur', title='Longueur (km)')))
graphRepartitionVehKm=alt.Chart(ResultatsEstimGeomCptInconnuGrpVehKm.groupby('type_estim').veh_km.sum().reset_index(), width=250, title='répartition du veh_km selon le type d\'estimation ').mark_bar().encode(
    x=alt.X('type_estim', axis=alt.Axis(labelAngle=0)), y='veh_km')

In [22]:
graphRepartitionTmja

alt.HConcatChart(...)

In [25]:
graphRepartitionVehKm

alt.Chart(...)

In [28]:
test=ResultatsEstimGeomCptInconnuGrpVehKm.groupby('type_estim').veh_km.sum().reset_index()
test.loc[0].veh_km/test.loc[1].veh_km
test.loc[2].veh_km/test.loc[1].veh_km

1.618909990120575

In [138]:
#analyse des prédiction par tranche de tmja avant recalage #attention : par secteur, sinon enlever la conidtion
lisGraphCompSolution=[alt.Chart(ResultatsEstimGeomCptConnu.loc[(ResultatsEstimGeomCptConnu.tmja<b)&(ResultatsEstimGeomCptConnu.tmja>a) & (ResultatsEstimGeomCptConnu.SEC=='16')]
              [['tmjo_2_sen','tmja_estime','estm_IA_1p','estm_IA_3p']], title=['Prédiction de trafic des points',f'entre {a} et {b} veh/j avant recalage']).transform_fold(
        ['tmja_estime','estm_IA_1p','estm_IA_3p'] ).mark_point().encode(
        x='value:Q',
        y='tmjo_2_sen:Q',
        tooltip='tmjo_2_sen',
        color='key:N')+alt.Chart(dfZone, width=200, height=200).mark_line(color='black').encode(x=alt.X('tmjo_2_sen', title='trafic calcule'), y=alt.Y('tmjo_2_sen',title='Trafic Mesure'))
 for a,b in ((0,50),(50,100),(100,200),(200,400),(400,1000),(1000,2000))]#,(2000,100000)
 
GraphCompSolution=alt.vconcat(*[alt.hconcat(*[lisGraphCompSolution[i+j] for i in range(2) if i+j<6]) for j in range(0,10,2)])
GraphCompSolution

alt.VConcatChart(...)

In [30]:
#une fois le choix du type de calcul effectué
#Au vu des éléments ci dessus en terme de répartiion sur ou sous-évaluation en fonction de la zone, on cree un resume de la fonction choisie selon le tmja calcule sans coeff
seuilTraficMethod=pd.read_csv(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\methode_selon_valeur_tmja.csv')

In [31]:
seuilTraficMethod

,SEC,seuil
0,1,200
1,2,400
2,3,400
3,4,400
4,5,mano
5,6,mano
6,7,400
7,8,400
8,9,400
9,10,ia


In [89]:
def calculTmjaFinal(tmja,tmja_estime,estm_IA_1p, seuil):
    """
    fonction de choix du tmja_final en fonction de la classe du tmja de base et des déduction stockées dans le fichier methode_selon_valeur_tmja.csv
    """
    if seuil=='ia' : 
        return estm_IA_1p
    elif seuil=='mano' : 
        return tmja_estime
    else :
        if tmja<int(seuil) : 
            return estm_IA_1p
        else : 
            return tmja_estime
#ResultatsEstimGeom=ResultatsEstimGeom.assign(SEC=seuilTraficMethod.SEC.astype(int)).merge(seuilTraficMethod, on='SEC')
ResultatsEstimGeom['tmja_calcule_final']=ResultatsEstimGeom.apply(lambda x : calculTmjaFinal(x['tmja'],x['tmja_estime'],x['estm_IA_1p'], x['seuil']), axis=1 )
ResultatsEstimGeom['veh_km_final']=ResultatsEstimGeom['tmja_calcule_final']*ResultatsEstimGeom.longueur/1000
dicoGraphApresRecalage={}
for s in ResultatsEstimGeom.SEC.unique() : 
    dfZone=ResultatsEstimGeom.loc[(ResultatsEstimGeom.SEC==s)].copy()
    base=alt.Chart(dfZone.loc[~dfZone['tmjo_2_sen'].isna()],title=f'zone EDA {s} apres recalage',width=250, height=200)
    points_corrige=base.mark_point(color='grey').encode(y='tmjo_2_sen:Q',x='tmja_calcule_final')
    ligne=base.mark_line(color='black').encode(x=alt.X('tmjo_2_sen', title='trafic calcule'), y=alt.Y('tmjo_2_sen',title='Trafic Mesure'))
    dicoGraphApresRecalage[s]=points_corrige+ligne
GraphApresRecalage=alt.vconcat(*[dicoGraphApresRecalage[str(i)] for i in range(15,20)])

In [48]:
#recalage des valeusr vraiment étrange : sur les valeurs sup à 5000 veh, c'est souvent bcp d'activites ou de logements, donc on prend la valeur max des deux
def correctionSup5000(nbDeplacementPop,nbDeplacementActivite) : 
    return max (nbDeplacementPop,nbDeplacementActivite)
ResultatsEstimGeom.loc[ResultatsEstimGeom.tmja_calcule_final>5000,'tmja_calcule_final']=ResultatsEstimGeom.loc[ResultatsEstimGeom.tmja_calcule_final>5000].apply(lambda x 
: correctionSup5000(x['nbDeplacementPop'],x['nbDeplacementActivite']), axis=1)

In [90]:
GraphVisuQuartile |GraphApresRecalage

alt.HConcatChart(...)

In [781]:
#export définitif
resultatsCat4HorsCpt=graph_filaire.assign(ident=graph_filaire.ident.astype(int)).merge(ResultatsEstimGeom.loc[ResultatsEstimGeom.tmjo_2_sen.isna()][['ident','tmja_calcule_final']])
resultatsCat4HorsCpt=resultatsCat4HorsCpt.rename(columns={'tmja_calcule_final':'tmjo_2_sen','geom':'geometry'}).set_geometry('geometry', crs='epsg:2154')
resultatsCat4HorsCpt['type_cpt']='estimation_catRhv4'
#resultatsCat4HorsCpt.to_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\affectationCat4HorsCpt.shp')
nonAffectes=graph_filaire.assign(ident=graph_filaire.ident.astype(int))
nonAffectes=nonAffectes.loc[~nonAffectes.ident.isin(resultatsComplet.ident.tolist())].rename(columns={'geom':'geometry'}).set_geometry('geometry', crs='epsg:2154')
nonAffectes=nonAffectes.assign(tmja_calcule_final=np.NaN,type_cpt='NC' )
#nonAffectes.to_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\non_affectes_final.shp')
resultatsComplet=pd.concat([gdf_traf_1234, resultatsCat4HorsCpt,nonAffectes], sort=False)
#export de travail
#ResultatsEstimGeom.to_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\affectationCat4HorsCpt_VTravail.shp')
#resultatsComplet.to_file(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\affect_trafic_voies_cat4\affectationComplete.shp')

## Chargement des resultats et visu
Charger directement les resultats

In [91]:
resultatComplet=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\FV_affectation_complete_grp1.shp')

In [92]:
resultatComplet

,ident,cat_rhv,groupe,tmjo_2_sen,type_cpt,id_comptag,longueur,nom_comm,nom_sect,veh_km,NOM_VOIE,geometry
0,8146,4,1.0,1370.0,ponctuel,"72, 73",0.133918,Bordeaux,intra-bld,183.467931,Rue François Leveque,"LINESTRING Z (417591.040 6424518.690 0.000, 41..."
1,7591,4,1.0,573.0,ponctuel,"850, 851",0.147489,Bordeaux,intra-bld,84.511220,Rue des Deux Ormeaux,"LINESTRING Z (415922.490 6422820.160 0.000, 41..."
2,19916,4,1.0,856.0,ponctuel,"184, 183",0.079545,Carbon-Blanc,extra-Rocade_rd,68.090118,Allée Calmette,"LINESTRING Z (422670.940 6427576.980 0.000, 42..."
3,20546,4,1.0,365.0,ponctuel,"1867, 1868",0.164048,Gradignan,extra-Rocade-Sud_rg,59.877680,Chemin de Mandavit,"LINESTRING Z (414842.000 6413435.580 0.000, 41..."
4,25302,4,1.0,595.0,ponctuel,"1476, 1477",0.371241,Bruges,extra-Rocade-NO_rg,220.888675,Rue du Commandant Mollière,"LINESTRING Z (415108.140 6428086.660 0.000, 41..."
5,35430,4,1.0,975.0,ponctuel,"1155, 1156",0.462702,Blanquefort,extra-Rocade-NO_rg,451.134433,Rue de Saint Ahon,"LINESTRING Z (413654.410 6431690.650 0.000, 41..."
6,19921,4,1.0,450.0,ponctuel,"910, 911",0.164789,Carbon-Blanc,extra-Rocade_rd,74.155206,Rue Gaston Cabannes,"LINESTRING Z (423636.370 6427679.340 0.000, 42..."
7,1236,4,1.0,1214.0,ponctuel,126,0.062514,Bègles,intra-Rocade-Sud_rg,75.892018,Rue Marcel Sembat,"LINESTRING Z (420006.830 6419556.380 0.000, 41..."
8,52232,4,1.0,1902.0,ponctuel,"1611, 1612",0.045454,Bègles,intra-Rocade-Sud_rg,86.454322,Rue Louis Denis Mallet,"LINESTRING Z (417998.740 6415851.410 0.000, 41..."
9,38414,4,1.0,40.0,ponctuel,241,0.009865,Le Haillan,extra-Rocade-NO_rg,0.394617,Rue de la Liberté,"LINESTRING Z (409198.250 6425785.670 0.000, 40..."
